In [1]:
import cobra
import GEMS
import pandas as pd
import numpy as np
import os
import re


import pickle
from etcpy import etc
from etcpy import tempDep
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

In [2]:
data = GEMS.load_exp_data('../data/ExpGrowth.csv')
path = '../'
params = pd.read_csv(os.path.join(path,'data/model_enzyme_params_new_tagged.csv'),index_col=0)
df = etc.calculate_thermal_params(params)

dfae_batch = data[0].set_index('Ts').rename_axis(None)
params

,Topt,Topt_std,Length,Tm,Tm_std,T90,dCpt,dCpt_std,topt_source,TmTag
P77467,306.15,10,262,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76077,311.15,10,309,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76078,310.15,10,95,328.717325,7.588735,NaN,-4000,1000,BullShit,Mean
P76079,309.15,10,248,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76081,310.15,10,356,328.717325,7.588735,NaN,-4000,1000,BullShit,Mean
...,...,...,...,...,...,...,...,...,...,...
P0AG80,313.15,10,438,329.166303,4.070000,330.45,-4000,1000,predicted,Exp
P77718,307.15,10,482,319.125751,4.070000,320.15,-4000,1000,predicted,Exp
P0A6D5,308.15,10,288,328.762604,7.710000,NaN,-4000,1000,predicted,Mean
standard,308.15,10,272,328.762604,7.710000,NaN,-4000,1000,BullShit,Mean


In [3]:
def aerobic(params):
    # thermalParams: a dictionary with ids like uniprotid_Topt 
    df = etc.calculate_thermal_params(params)
    #mae = pickle.load(open(os.path.join(path,'models/aerobic.pkl'),'rb'))
    mae = cobra.io.load_matlab_model('../models/ecModel.mat')
    
    match = re.compile('usage_prot_*', re.IGNORECASE)
    for react in mae.reactions:
        if match.match(react.id) and react.lower_bound != -1000:
            ori_lb = react.lower_bound
            react.lower_bound = ori_lb*2.5


    mae.reactions.EX_glc__D_e.lower_bound = -100
    mae.reactions.BIOMASS_Ec_iJO1366_core_53p95M.upper_bound = 50
    #mae.medium['EX_glc__D_e'] = 100
    
    try: 
        rae, gae, oae = etc.simulate_growth(mae,dfae_batch.index+273.15,df=df,sigma=0.5)
    except: 
        rae = np.zeros(dfae_batch.shape[0])
        model = None
    
    rae = [0 if x is None else x for x in rae]
    rae = [0 if x<1e-3 else x for x in rae]
    rexp = dfae_batch['r'].values
    #print(rae)
    print(abs(rae - dfae_batch['r'].values))
    print('r2_batch:',r2_score(rexp,rae))
    print('MSE_ae',MSE(rexp,rae))
    return {'growth':np.array(rae), 'glucose':np.array(gae), 'oxygen': np.array(oae)}, r2_score(rexp,rae)

In [4]:
S = 0
SMin = 0
df = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux'])
for i in range(1,101):
    print(i)
    newParams = tempDep.sample_data_uncertainty_with_constraint_increasing_topt(params, ['Tm'])
    rae, score = aerobic(newParams)

    if score > S:
        S = score
        newParams.to_csv('../data/BestParamsTm.csv')
        bestParams = newParams
    
        fig, ax1 = plt.subplots(figsize=(9, 6))
        ax2 = ax1.twinx()

        ax1.scatter(dfae_batch.index,dfae_batch['r'].values,label='Experimental Growth',color='k')
        ax1.plot(dfae_batch.index,rae['growth'],markersize=5,marker='o',label='Pred Growth')

        ax2.plot(dfae_batch.index,rae['glucose'],markersize=5,marker='o',label='Glucose Uptake', color='tab:brown')
        ax2.plot(dfae_batch.index,rae['oxygen'],markersize=5,marker='o',label='Oxygen Uptake', color='tab:gray')

        ax1.set_xlabel("Temperature ($^\circ$C)")
        ax1.set_ylabel("Aerobic $r_{max}$ (h$^{-1}$)")

        ax2.set_ylabel("Uptake fkux")

        fig.legend(fontsize=10, loc=2)
        fig.tight_layout()
        fig.savefig("../results/TmWigFirst" + str(i) + ".pdf")
        plt.close(fig)
    elif score < SMin:
        SMin = score
        newParams.to_csv('../data/WorstParams.csv')
    
        fig, ax1 = plt.subplots(figsize=(9, 6))
        ax2 = ax1.twinx()

        ax1.scatter(dfae_batch.index,dfae_batch['r'].values,label='Experimental Growth',color='k')
        ax1.plot(dfae_batch.index,rae['growth'],markersize=5,marker='o',label='Pred Growth')

        ax2.plot(dfae_batch.index,rae['glucose'],markersize=5,marker='o',label='Glucose Uptake', color='tab:brown')
        ax2.plot(dfae_batch.index,rae['oxygen'],markersize=5,marker='o',label='Oxygen Uptake', color='tab:gray')

        ax1.set_xlabel("Temperature ($^\circ$C)")
        ax1.set_ylabel("Aerobic $r_{max}$ (h$^{-1}$)")

        ax2.set_ylabel("Uptake fkux")

        fig.legend(fontsize=10, loc=2)
        fig.tight_layout()
        fig.savefig("../results/TmWigFirst" + str(i) + ".pdf")
        plt.close(fig)
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'grwoth', 'o2flux', 'gluFlux'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df = pd.concat([df, df1])
if 'bestParams' not in globals():
    bestParams = newParams

1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-29
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.14421134005815914
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.31416725641828125
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.7248309043418802
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  1.133020918656435
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  1.4330267546434552
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  1.5231737545208441
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.431275192663121
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.263472177934689
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.1369473095440024
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.028125908814962
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  0.8949592540948936
NGAM is: 6.9000236503978725
Growth at  44.45349109757

In [5]:
df.to_csv('../results/modelResultsTm.csv')

In [6]:
#S = 0
#SMin = 0
#df = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux'])
for i in range(1,21):
    print(i)
    if i == 1:
        bestParams = tempDep.sample_data_uncertainty_with_constraint_increasing_topt(bestParams, ['Topt', 'dCpt'])
        rae, score = aerobic(bestParams)

    else:
        bestParams = tempDep.sample_data_uncertainty_with_constraint_small_increasing_topt(bestParams, ['Topt', 'dCpt'])
        rae, score = aerobic(bestParams)
    
    if score > S:
        S = score
        bestParams.to_csv('../data/BestParamsTopt.csv')
    
        fig, ax1 = plt.subplots(figsize=(9, 6))
        ax2 = ax1.twinx()

        ax1.scatter(dfae_batch.index,dfae_batch['r'].values,label='Experimental Growth',color='k')
        ax1.plot(dfae_batch.index,rae['growth'],markersize=5,marker='o',label='Pred Growth')

        ax2.plot(dfae_batch.index,rae['glucose'],markersize=5,marker='o',label='Glucose Uptake', color='tab:brown')
        ax2.plot(dfae_batch.index,rae['oxygen'],markersize=5,marker='o',label='Oxygen Uptake', color='tab:gray')

        ax1.set_xlabel("Temperature ($^\circ$C)")
        ax1.set_ylabel("Aerobic $r_{max}$ (h$^{-1}$)")

        ax2.set_ylabel("Uptake fkux")

        fig.legend(fontsize=10, loc=2)
        fig.tight_layout()
        fig.savefig("../results/TmWigFirstTopt" + str(i) + ".pdf")
        plt.close(fig)
    elif score < SMin:
        SMin = score
        bestParams.to_csv('../data/WorstParamsTopt.csv')
    
        fig, ax1 = plt.subplots(figsize=(9, 6))
        ax2 = ax1.twinx()

        ax1.scatter(dfae_batch.index,dfae_batch['r'].values,label='Experimental Growth',color='k')
        ax1.plot(dfae_batch.index,rae['growth'],markersize=5,marker='o',label='Pred Growth')

        ax2.plot(dfae_batch.index,rae['glucose'],markersize=5,marker='o',label='Glucose Uptake', color='tab:brown')
        ax2.plot(dfae_batch.index,rae['oxygen'],markersize=5,marker='o',label='Oxygen Uptake', color='tab:gray')

        ax1.set_xlabel("Temperature ($^\circ$C)")
        ax1.set_ylabel("Aerobic $r_{max}$ (h$^{-1}$)")

        ax2.set_ylabel("Uptake fkux")

        fig.legend(fontsize=10, loc=2)
        fig.tight_layout()
        fig.savefig("../results/TmWigFirstTopt" + str(i) + ".pdf")
        plt.close(fig)
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'grwoth', 'o2flux', 'gluFlux'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df = pd.concat([df, df1])

1
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.1940131696850393
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.30225478340143064
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.4826097833548562
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.6171355567806539
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.7601821451197844
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.9123205422475943
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.1229196098909506
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.4664252940924944
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.5042418827380186
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4337765206006063
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.3232955698127364
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.1582135786710386
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:

/home/aditi/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.011028294592563066
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.036643738929371276
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.06977832292306552
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.11244676268154899
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.15823029485852075
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.23446319076613162
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  0.3980040159039303
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  0.5513578467934239
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  0.7096658747132174
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  0.9086876518196186
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.0587308317591373
NGAM is: 6.974987953458825
Growth at  45.28989924561603 i

/home/aditi/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.005487955872491679
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.01971424456298765
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.035909510180782214
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.05989227649024413
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.0939735825076567
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.1575459410418864
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  0.3036091073461757
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  0.4266934251460769
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  0.5227332630239069
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  0.6563296846159351
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  0.8185429025824092
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:

/home/aditi/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.002531105674839939
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.009400672823509652
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.01824780880908898
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.03212703708552346
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.05286161881079968
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.09356893101883616
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  0.20080226611258775
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  0.32029243638931515
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  0.45878342539329503
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  0.6874932211047364
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  0.8517415326690515
NGAM is: 6.974987953458825
Growth at  45.2898992456160

In [7]:
df.to_csv('../results/modelResultsTopt.csv')

In [8]:
df

,iter,temp,growth,o2flux,gluFlux,grwoth
0,1,20.153299,0.144211,4.965558,1.995870,NaN
1,1,23.001511,0.314167,8.631334,3.763005,NaN
2,1,26.331896,0.724831,18.726097,8.535223,NaN
3,1,28.251820,1.133021,28.430026,13.693786,NaN
4,1,29.994212,1.433027,39.264791,31.691585,NaN
...,...,...,...,...,...,...
10,20,42.238764,0.687493,19.465052,17.746253,NaN
11,20,44.453491,0.851742,23.130265,18.905936,NaN
12,20,45.289899,0.911676,24.538288,19.096327,NaN
13,20,46.645331,1.011201,27.310952,22.224610,NaN
